# Modellierung von verteilten Systemen

Durch die Trends Microservices und Serverless sowie die häufigere Verwendung von Datenbanken, die aufgrund bestimmter Performance-Anforderungen nicht mehr über die traditionellen Konsistenzgarantien verfügen, werden sich Entwicklerteams immer häufiger mit den Problemen von verteilten Systemen auseinander setzen müssen. Zu diesen Problemen gehört auch, wie man Fehler über Servicegrenzen hinweg debuggen kann, oder wie es ermöglicht werden kann, dass die verschiedenen Services unabhängig voneinander deployt werden können. Dieser Artikel jedoch befasst sich mit zwei speziellen Fehlerkategorien, welche im besonderen Maß durch verteilte Systeme provozierten werden: Konsistenzfehler und Nebenläufigkeitsfehler.

Diese Arten von Fehlern sind besonders schwierig zu reproduzieren, können unbemerkt zu Datenkorruption führen und sind mit den etablierten Testvorgehen kaum zu identifiziert.

In der agilen Softwareentwicklung wird ein hohes Maß an technischer Qualität des Softwareinkrements gefordert. Dazu gehört insbesondere, dass die Software automatisiert auf der Ebene des Source-Codes (Unit Tests), im Verbund von Komponenten (Integration Tests) und aus der Perspektive des Kunden (Acceptance Tests) getestet wird. Der Umfang der jeweiligen Testkategorie wird anhand der Testpyramide verdeutlicht:

![Test Pyramide](https://www.borisgloger.com/wp-content/uploads/blog/Testp-300x281-300x281.jpeg "Test Pyramide")

Es wurde die Form einer Pyramide gewählt wurde, weil der Großteil der Tests eines Systems aus Unit Tests bestehen sollte. Entsprechen die Unit Tests den FIRST-Kriterien, so liefern Unit Tests am schnellsten und zuverlässigsten Feedback. Zwar liefern Integrationsstests und besonders Akzeptanztests am langsamsten Feedback, dafür 

Zwar liefern Akzeptanztests am langsamsten Feedback und ihre Pflege gestaltet sich am aufwändigsten, dafür sind die Tests am nächsten zur Realität. Ein Akzeptanztests greift auf die gleichen Testmöglichkeiten wir ein menschlicher Tester oder ein Benutzer des Systems zurück.

Integrationsstests stellen einen Mittelweg zwischen Unit Tests und Akzeptanztests dar. Mit Integrationsstest testet man das technische Zusammenspiel von Komponenten oder Services, jedoch nicht über die Benutzeroberfläche des Systems, sondern ähnlich wie bei Unit Tests in einem künstlich erzeugten Test-Setup. Das Setup ist weniger künstlich als bei Unit Tests, jedoch nicht so realitätsnah wie bei Akzeptanztest.

Der Begriff Testpyramide spiegelt den breiten Konsens innerhalb der agilen Entwicklungscommunity wieder, dass der beste Kompromiss zwischen Aufwand und Ertrag von Tests zu einer Verteilung von Unit Tests zu Integrationstests und Akzeptanztests zu der Form einer Pyramide führt.

Damit Unit Tests schnell und zuverlässig Feedback liefern, sollten sie den FIRST-Kriterien entsprechen:

* Fast: Unit Tests sollen schnell sein
* Isolated: Unit Tests sollten unabhängig von externen Abhängigkeiten wie einer Datenbank sein
* Repeatable: Unit Tests sollen wiederholbar, also deterministisch sein
* Self Validating: Am Ende eines Unit Tests sollte dieser ein eindeutiges Ergebnis zurückliefern
* Timely: Man sollte Unit Tests möglichst früh schreiben, am besten bereits vor dem Produktivcode

Durch die FIRST-Kriterien wird klar, dass Unit Tests sowohl für Konsistenzfehler als auch für Nebenläufigkeitsfehler das falsche Werkzeug sind. Angenommen, man möchte ein System, das eine Datenbank mit dem Konsistenzmodell “Eventual Consistency” nutzt, auf Konsistenzfehler testen. Da ein Unit Test, aufgrund des Isolated-Kriteriums, keine Abhängigkeit zu einer Datenbank haben darf, kann das System nicht in Zusammenspiel mit der Datenbank in einem Unit Test geprüft werden. Der Workaround, stattdessen eine einer Attrappe der Datenbank zu verwenden, würde nur zu einem Gefühl der falschen Sicherheit verleiten, da das Verhalten der Datenbank in vielerlei Hinsicht nicht exakt nachgestellt werden kann. Auch zum Testen von Nebenläufigkeit sind Unit Tests eher ungeeignet, treten Nebenläufigkeitsprobleme doch vor allem zufällig und indeterministisch auf. Genau diese Eigenschaften schließt das Repeatable-Kriterium aber aus.

Sowohl für Integrationstests als auch noch stärker für Akzeptanztest gilt, dass auf dieser Ebene die Kombination der möglichen Setups, Schritte und Konstellationen zu so einer hohen Zahl an Testkombinationen führt, dass die Wahrscheinlichkeit gering ist, alle Konsistenz- und Nebenläufigkeitsfehler zu entdecken.

## Modellierung als Alternative zur Testpyramide

Eine Alternative zum Testen mit den Mitteln der Testpyramide stellt die Modellierung des verteilten oder nebenläufigen Systems dar. Das bedeutet, man abstrahiert von dem konkreten Code und konzentriert sich auf das Verhalten und die Eigenschaften des Systems. Dazu stellt man folgende Fragen:
Welche Events können auftreten?
Wie ist der Zustand des Systems nach einem bestimmten Event?
Welche Bedingung muss das System schlussendlich und welche Bedingungen jederzeit erfüllen?
Die Antworten auf diese Fragen bilden ein Modell des zu testenden Systems. Ein Modellprüfer kann damit jegliche Kombination von Events generieren und automatisiert testen, ob nach jedem Event die erforderlichen Bedingungen an den Zustand des Systems erfüllt sind. Findet der Modellprüfer eine Verletzung einer Bedingung, so hat man einen logischen Fehler im Design des Systems gefunden. Findet der Modellprüfer keinen Fehler, so erfüllt das Design des Systems die formulierten Bedingungen und die Übersetzung des Designs in die gewünschte Programmiersprache und Laufzeitumgebung kann beginnen. Da bei der Implementierung des Designs Übersetzungsfehler passieren könnten, die das geprüfte Modell verletzen würden, kann man auf die Werkzeuge der Testpyramide weiterhin nicht verzichten.

Um den Nutzen und die konkrete Anwendung von Modellierung im Zusammenspiel mit einem Modellprüfer zu demonstrieren, werden im folgenden zwei Beispiele behandelt. Das erste Beispiel überprüft ein verteiltes System auf Konsistenzverletzungen, das zweite Beispiel zeigt auf, wie man Nebenläufigkeitsfehler identifizieren und ausschließen kann.

## Beispiel 1: Konsistenz im verteilten System

Als Beispiel für ein verteiltes Systems soll ein digitales Sparschwein mit Buchhalterfunktion für Familien dienen. Um das Beispiel möglichst einfach zu halten, muss das Sparschwein ausschließliche die Einzahlung und die Abhebung von 1-Euro-Münzen unterstützen. Jede Einzahlung und Abhebung wird als Event in einem Dokument einer Dokumentendatenbank festgehalten. Dadurch können alle Ein- und Auszahlungen des Sparschweins nachvollzogen werden. Es darf nicht möglich sein, dass ein Nutzer einen Euro abhebt, obwohl keine Münzen mehr im Sparschwein sind. Die Prüfung dieser Bedingung soll über eine andere Datenbank erfolgen, welche nur den aktuellen Betrag innerhalb des Sparschweins festhält.

Wie eingangs erwähnt, sind folgende Fragen zu beantworten:

1. Welche Events können auftreten?
1. Wie ist der Zustand des Systems nach einem bestimmten Event?
1. Welche Bedingung muss das System schlussendlich und welche Bedingungen jederzeit erfüllen?

Im folgendem starten wir den Modellierungsprozess und finden Antworten auf diese 3 Fragen.

### Welche Events können auftreten?

Um die erste Frage zu beantworten, sind wir bereits gezwungen, das System auf einer hohen Abstraktionsebene in Schritte zu modellieren. Dadurch erkennen wir auch, welche Schritte atomar sind und welche nicht. Beispielsweise können wir nicht gleichzeitig aus einer Dokumentendatenbank lesen und gleichzeitig in diese schreiben. Würde man allein diesen Modellierungsschritt gehen, ließen sich bereits dadurch viele Fehler im Design eines verteilten Systems vermeiden.

In unserem vorliegenden Fall kann man die Events aus den Schritten ableiten, die ein mögliches Buchhaltersparschweinsystem durchführen würde.



Für jede Transaktionsanfrage eines Nutzers werden diese Schritte ausgeführt. Das System hat keinen Einfluss zu welchem Zeitpunkt Transaktionsanfragen an das System gestellt werden. Das System bestimmt aber, welche Schritte danach ausgeführt werden, in welcher Reihenfolge und ob dies synchron oder asynchron geschieht. Diese Festlegungen bezeichnen wir als "Event-Modell”.

* Der Inhalt des Sparschweins wurde ausgelesen
* Familienmitglieder haben die Anfrage zur Einzahlung eines Euros gestellt
* Familienmitglieder haben die Anfrage zur Abhebung eines Euros gestellt
* Aus der Dokumentendatenbank wurde das aktuelle Dokument ausgelesen
* Eine Transaktion (Abhebung oder Einzahlung) wurde festgehalten
* Der Betrag des Sparschweins wurde aktualisiert

Wir modellieren die Events absichtlich als Ereignisse, die erfolgreich stattgefunden haben. Später wird klarer werden, warum wir die Events nicht als Anweisungen wie z.B. “Halte Transaktion fest” modellieren.

Um diese Events als Datenstrukturen zu modellieren, nutze ich die Programmiersprache Clojure. Diese Sprache verfügt über nur wenig Syntax und man kann Datenstrukturen kurz und prägnant darstellen. Funktionsaufrufe in Clojure folgen der Prefix-Notation. Statt



In [1]:
(do (require '[clojupyter.misc.helper :as helper])
    (require '[cemerick.pomegranate :as pom])
    (helper/add-dependencies '[org.clojure/test.check "0.9.0"])
    (helper/add-dependencies '[org.clojure/math.combinatorics "0.1.4"])
    (helper/add-dependencies '[progrock "0.1.2"])
    (helper/add-dependencies '[me.lomin/sayang "0.3.0"])
    (pom/add-classpath "src")
    (require '[me.lomin.piggybank.accounting.doc :as a-doc]))

nil

In [3]:
(a-doc/accounting-events)

[:balance-write {:amount 1, :process-id 0} :doc "Das Saldo des Sparschweins wurde aktualisiert."]

In [4]:
(a-doc/print-source a-doc/accounting-events)

(defn accounting-events []
  [:process
   {:amount 1, :process-id 0}
   :doc
   "Eine Anfrage mit der Prozess-Id 0 zur Einzahlung von 1 Euro wurde gestartet."]
  [:process
   {:amount -1, :process-id 1}
   :doc
   "Eine Anfrage mit der Prozess-Id 1 zur Abhebung von 1 Euro wurde gestartet."]
  [:accounting-read
   {:amount 1, :process-id 0}
   :doc
   "Die notwendigen Daten aus dem Buchhaltungssystem wurden abgefragt. Das Ergebnis der Abfrage wurde abgelegt. Ausgelöst wurde dieses Event durch den Prozess mit der Prozess-Id 0."]
  [:accounting-write
   {:amount 1, :process-id 0}
   :doc
   "Die Transaktion wurde im Buchhaltungssystem festgehalten."]
  [:balance-write
   {:amount 1, :process-id 0}
   :doc
   "Das Saldo des Sparschweins wurde aktualisiert."])


nil